In [1]:
import bo_methods_lib
from bo_methods_lib.CS2_bo_functions_multi_dim import LHS_Design, test_train_split
from bo_methods_lib.CS2_create_data import create_y_data, calc_y_exp, create_sse_data, calc_muller
from bo_methods_lib.bo_functions_generic import clean_1D_arrays
from itertools import combinations_with_replacement
from itertools import combinations
from itertools import permutations

from scipy.stats import qmc
import torch
import numpy as np
import pandas as pd
import os

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#LHS Design in python - May switch code to use this instead for conciseness
emulator = True
exp_LHS = False
emulator = False
seed = 9
# LHS_Points = 100
#20, 40, 200, 600, 1000
LHS_Points =20
LHS_xtra2 = np.round(np.array([[-1.174050,  -1.657363, -9.801574, -1.555507,  0.656785,  0.016924, 14.569803, -0.183648],
                               [-0.917405, -0.685736, -1.130157,  1.924449, -1.374322,-0.378308,14.30698,1.701635]]),6)
same_LHS_pts = 5
d = 8
Bound_Cut = True
denseX = True

if Bound_Cut == True:
    cut_bounds = '_cut_bounds'
else:
    cut_bounds = ""
    
if denseX == True:
    dense = "_dense"
else:
    dense = ""
    
        
# d = 2
sampler = qmc.LatinHypercube(d=d, seed = seed)
sample = sampler.random(n=LHS_Points-same_LHS_pts)

#Only going to regress "a and b" parameters
skip_param_types = 1 #(Skiiping A)
l_bounds =  [-2, -2, -10, -2, -2, -2,  5, -2]
u_bounds =  [ 2,  2,   0,  2,  2,  2, 15,  2]

# l_bounds =  [-2, -2, -10, -2]
# u_bounds =  [ 2,  2,   0,  2]

# l_bounds =  [-2, -2]
# u_bounds =  [2, 2 ]
LHS = qmc.scale(sample, l_bounds, u_bounds)

sampler = qmc.LatinHypercube(d=d, seed = seed)
sample = sampler.random(n=LHS_Points)

sampler_xtra = qmc.LatinHypercube(d=d, seed = seed + 1)
sample_xtra = sampler_xtra.random(n=same_LHS_pts-len(LHS_xtra2))
LHS_xtra = qmc.scale(sample_xtra, l_bounds, u_bounds)
LHS_xtra = np.round(LHS_xtra, 6)

LHS_xtra = np.vstack((LHS_xtra, LHS_xtra2))
print(LHS_xtra)

LHS = np.vstack((LHS, LHS_xtra))
print(LHS.shape)

[[ 0.725331  1.723091 -9.42815   0.467624 -0.017073 -0.847893  6.036545
   0.87767 ]
 [-1.234012 -0.609235 -6.084443 -1.11762  -1.434347 -0.337736 12.242987
  -0.577918]
 [ 0.47334  -1.660774 -0.464505  0.791295  1.698514  0.862347 10.645607
  -1.959774]
 [-1.17405  -1.657363 -9.801574 -1.555507  0.656785  0.016924 14.569803
  -0.183648]
 [-0.917405 -0.685736 -1.130157  1.924449 -1.374322 -0.378308 14.30698
   1.701635]]
(20, 8)


In [3]:
#LHS for experimental data points
exp_LHS_Points = 20 #0 , 10, 15, 20, or 25 for 4, 8, 12, 16, and 24 params
exp_d = 2
exp_sampler = qmc.LatinHypercube(d=exp_d, seed = seed)
exp_sample = exp_sampler.random(n=exp_LHS_Points)
exp_sample_corner = exp_sampler.random(n=exp_LHS_Points)

if Bound_Cut == True:
    exp_l_bounds =  [-1.0, 0.0]
    exp_u_bounds =  [0.5 , 1.5]
else:
    exp_l_bounds =  [-1.5, -0.5]
    exp_u_bounds =  [1 , 2]

exp_l_bounds_corner =  [0, 1]
exp_u_bounds_corner =  [1 , 2]

if exp_LHS == True:
    exp_pts = qmc.scale(exp_sample, exp_l_bounds, exp_u_bounds)
    exp_LHS_corner = qmc.scale(exp_sample_corner, exp_l_bounds_corner, exp_u_bounds_corner)
    
    #Add Extra experimental data from corner LHS
    # exp_pts = np.concatenate((exp_LHS, exp_LHS_corner))

    # print(exp_pts)
    # print(exp_LHS_corner)
else:
    #Create a linspace for the number of dimensions
    X1 = np.linspace(exp_l_bounds[0], exp_u_bounds[0], 5)
    X2 = np.linspace(exp_l_bounds[1], exp_u_bounds[1], 5)
    exp_pts = np.array(np.meshgrid(X1, X2)).T.reshape(-1, len(exp_l_bounds))

print(exp_pts)

[[-1.     0.   ]
 [-1.     0.375]
 [-1.     0.75 ]
 [-1.     1.125]
 [-1.     1.5  ]
 [-0.625  0.   ]
 [-0.625  0.375]
 [-0.625  0.75 ]
 [-0.625  1.125]
 [-0.625  1.5  ]
 [-0.25   0.   ]
 [-0.25   0.375]
 [-0.25   0.75 ]
 [-0.25   1.125]
 [-0.25   1.5  ]
 [ 0.125  0.   ]
 [ 0.125  0.375]
 [ 0.125  0.75 ]
 [ 0.125  1.125]
 [ 0.125  1.5  ]
 [ 0.5    0.   ]
 [ 0.5    0.375]
 [ 0.5    0.75 ]
 [ 0.5    1.125]
 [ 0.5    1.5  ]]


In [4]:
#Creates Experimental data and saves to a csv
# Constants = np.array([-200,-100,-170,15,   #A
#                       -1,-1,-6.5,0.7,    #a
#                       0,0,11,0.6,        #b
#                       -10,-10,-6.5,0.7,  #c
#                       1,0,-0.5,-1,      #x0
#                       0,0.5,1.5,1])     #yo

Constants = np.array([[-200,-100,-170,15],
                      [-1,-1,-6.5,0.7],
                      [0,0,11,0.6],
                      [-10,-10,-6.5,0.7],
                      [1,0,-0.5,-1],
                      [0,0.5,1.5,1]])

# Theta_True = np.array([-2,2])
# Xexp = np.array([-2,-1,0,1,2])
# Xexp = np.array([[-0.558,-0.050,0.623,-0.82, 0.22],
#                  [1.442,0.467,0.028,0.62,0.30]]) #This one doesn't have corner points

Xexp = np.array([[-0.558,-0.050,0.623,-0.82, 0.22],
                 [1.442,0.467,0.028,0.62,0.30]])

Xexp = np.vstack((exp_pts, Xexp.T))
print(Xexp[0])
print(Xexp.shape)

n = Xexp.shape[0]
print(Xexp)
noise_std = 0.01
# noise_std = 0
Yexp = calc_y_exp(Constants, Xexp, noise_std)
print(Yexp)

[-1.  0.]
(30, 2)
[[-1.     0.   ]
 [-1.     0.375]
 [-1.     0.75 ]
 [-1.     1.125]
 [-1.     1.5  ]
 [-0.625  0.   ]
 [-0.625  0.375]
 [-0.625  0.75 ]
 [-0.625  1.125]
 [-0.625  1.5  ]
 [-0.25   0.   ]
 [-0.25   0.375]
 [-0.25   0.75 ]
 [-0.25   1.125]
 [-0.25   1.5  ]
 [ 0.125  0.   ]
 [ 0.125  0.375]
 [ 0.125  0.75 ]
 [ 0.125  1.125]
 [ 0.125  1.5  ]
 [ 0.5    0.   ]
 [ 0.5    0.375]
 [ 0.5    0.75 ]
 [ 0.5    1.125]
 [ 0.5    1.5  ]
 [-0.558  1.442]
 [-0.05   0.467]
 [ 0.623  0.028]
 [-0.82   0.62 ]
 [ 0.22   0.3  ]]
[  23.46640693  -17.00373675  -57.52642398  -91.13045667  -15.60795062
    6.79732697  -42.66124149  -31.04841453  -87.26032365 -131.52037117
  -21.07934865  -68.56222422  -30.04575585    5.70612482  -80.07354755
  -63.79007524  -75.64064679  -20.92949814   37.62773215   47.30870824
 -102.82678081  -50.5141435    18.20105864   80.41225424  135.11609314
 -146.69947812  -80.76773816 -108.16663897  -40.67072465  -72.22508357]


In [5]:
df = pd.DataFrame({'X1exp': Xexp[:, 0], 'X2exp': Xexp[:, 1], 'Yexp':Yexp})
# df = pd.DataFrame({'Xexp': Xexp, 'Yexp':Yexp})
print(df)

path = 'Input_CSVs/Exp_Data/d='+str(exp_d)
if not os.path.exists(path):
    os.makedirs(path)
df.to_csv(os.path.join(path,'n='+str(n)+cut_bounds+dense+'.csv'))
# df.to_csv(os.path.join(path,'n='+str(n)+cut_bounds+'.csv'))
# df.to_csv(os.path.join(path,'n='+str(n)+'_no_noise.csv'))

    X1exp  X2exp        Yexp
0  -1.000  0.000   23.466407
1  -1.000  0.375  -17.003737
2  -1.000  0.750  -57.526424
3  -1.000  1.125  -91.130457
4  -1.000  1.500  -15.607951
5  -0.625  0.000    6.797327
6  -0.625  0.375  -42.661241
7  -0.625  0.750  -31.048415
8  -0.625  1.125  -87.260324
9  -0.625  1.500 -131.520371
10 -0.250  0.000  -21.079349
11 -0.250  0.375  -68.562224
12 -0.250  0.750  -30.045756
13 -0.250  1.125    5.706125
14 -0.250  1.500  -80.073548
15  0.125  0.000  -63.790075
16  0.125  0.375  -75.640647
17  0.125  0.750  -20.929498
18  0.125  1.125   37.627732
19  0.125  1.500   47.308708
20  0.500  0.000 -102.826781
21  0.500  0.375  -50.514143
22  0.500  0.750   18.201059
23  0.500  1.125   80.412254
24  0.500  1.500  135.116093
25 -0.558  1.442 -146.699478
26 -0.050  0.467  -80.767738
27  0.623  0.028 -108.166639
28 -0.820  0.620  -40.670725
29  0.220  0.300  -72.225084


In [6]:
#Array for Standard Approach
all_p_standard = pd.DataFrame(LHS, columns = ["a1", "a2", "a3", "a4", "b1", "b2", "b3", "b4"])
# all_p_standard = pd.DataFrame(LHS, columns = ["a1", "a2", "a3", "a4"])
# all_p_standard = pd.DataFrame(LHS, columns = ["theta1", "theta2"])

print(Xexp,Xexp.shape)
#Array for Emulator approach
df = all_p_standard.iloc[np.repeat(np.arange(len(all_p_standard)), n)]
df = df.reset_index(drop = True)
# Xexp_repeat = np.repeat(Xexp, LHS_Points, axis = 1).reshape(Xexp.shape[1],-1)
# print(Xexp_repeat.reshape(300,2))
Xexp_repeat = np.tile(Xexp, (len(all_p_standard),1))
#Add Xexp 
for i in range(Xexp.shape[1]):
    df['X'+str(i+1)+'exp'] = Xexp_repeat[:,i]    
all_p_emulator = df

print(all_p_emulator.shape)
# print(all_p_standard)


[[-1.     0.   ]
 [-1.     0.375]
 [-1.     0.75 ]
 [-1.     1.125]
 [-1.     1.5  ]
 [-0.625  0.   ]
 [-0.625  0.375]
 [-0.625  0.75 ]
 [-0.625  1.125]
 [-0.625  1.5  ]
 [-0.25   0.   ]
 [-0.25   0.375]
 [-0.25   0.75 ]
 [-0.25   1.125]
 [-0.25   1.5  ]
 [ 0.125  0.   ]
 [ 0.125  0.375]
 [ 0.125  0.75 ]
 [ 0.125  1.125]
 [ 0.125  1.5  ]
 [ 0.5    0.   ]
 [ 0.5    0.375]
 [ 0.5    0.75 ]
 [ 0.5    1.125]
 [ 0.5    1.5  ]
 [-0.558  1.442]
 [-0.05   0.467]
 [ 0.623  0.028]
 [-0.82   0.62 ]
 [ 0.22   0.3  ]] (30, 2)
(600, 10)


In [7]:
## Create Training Data and Testing Data
#Set noise parameters and load csv file to generate training data
# all_p = LHS_Design("LHS_Toy_3_Input.csv")

#Create y or SSE data
#Save data
if emulator == False:
    all_2_data = all_p_standard.copy()
    all_2_in_y = create_sse_data(all_p_standard, Xexp, Yexp, Constants, obj = "obj", skip_param_types=skip_param_types)
    all_2_data['Yexp'] = all_2_in_y
#     
    all_2_data_ln_obj = all_p_standard.copy()
    all_2_in_y_ln_obj = create_sse_data(all_p_standard, Xexp, Yexp, Constants, obj = "LN_obj", skip_param_types=skip_param_types)
    all_2_data_ln_obj['Yexp'] = all_2_in_y_ln_obj
    
    titles_all = np.array(['all_st_data','all_st_ln_obj_data'])
    len_data = str(len(all_2_data))
    all_array = [all_2_data,all_2_data_ln_obj]

else:
    all_3_data = all_p_emulator.copy()
    all_3_in_y = create_y_data(all_p_emulator, Constants, Xexp, skip_param_types=skip_param_types)
    all_3_data['Yexp'] = all_3_in_y
    
    titles_all = np.array(['all_emul_data'])
    len_data = str(len(all_3_data))
    all_array = [all_3_data]
    
# print(all_array)

# Save Train Data/Test Data
for i in range(len(titles_all)):
    path = 'Input_CSVs/Train_Data/d=' + str(d) +"/"+ titles_all[i]+"/"
    if not os.path.exists(path):
        os.makedirs(path)
    all_array[i].to_csv(os.path.join(path,'t='+len_data + cut_bounds+dense)+ '.csv')